In [1]:
# import the necessary packages
from scipy.spatial import distance as dist
from collections import OrderedDict
import numpy as np
 
class CentroidTracker():
    def __init__(self, maxDisappeared=100):
        self.nextObjectID = 0
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()
        self.maxDisappeared = maxDisappeared
        
    def register(self, centroid):
        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        self.nextObjectID += 1
        
    def deregister(self, objectID):
        del self.objects[objectID]
        del self.disappeared[objectID]
        
    def update(self, rects):
        if len(rects) == 0:
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)
            return self.objects
        
        # initialize an array of input centroids for the current frame
        inputCentroids = np.zeros((len(rects), 2), dtype="int")

        # loop over the bounding box rectangles
        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            # use the bounding box coordinates to derive the centroid
            cX = int((startX + endX) / 2.0)
            cY = int((startY + endY) / 2.0)
            inputCentroids[i] = (cX, cY)
            
        # if we are currently not tracking any objects take the input
        # centroids and register each of them
        if len(self.objects) == 0:
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i])
        
        else:
            # grab the set of object IDs and corresponding centroids
            objectIDs = list(self.objects.keys())

            objectCentroids = list(self.objects.values())
            
            D = dist.cdist(np.array(objectCentroids), inputCentroids)
            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]
            usedRows = set()
            usedCols = set()
            for (row, col) in zip(rows, cols):
            
                if row in usedRows or col in usedCols:
                    continue

                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.disappeared[objectID] = 0
 
                # indicate that we have examined each of the row and
                # column indexes, respectively
                usedRows.add(row)
                usedCols.add(col)
                
            # compute both the row and column index we have NOT yet
            # examined
            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)
            
            if D.shape[0] >= D.shape[1]:
                # loop over the unused row indexes
                for row in unusedRows:
                    # grab the object ID for the corresponding row
                    # index and increment the disappeared counter
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1
 
                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)
                        
            else:
                for col in unusedCols:
                    self.register(inputCentroids[col])
 
        # return the set of trackable objects
        return self.objects

In [2]:
from scipy.spatial import distance as dist
import numpy as np
np.random.seed(42)
objectCentroids = np.random.uniform(size=(2, 2))
centroids = np.random.uniform(size=(3, 2))
D = dist.cdist(objectCentroids, centroids) # Euclidean distance between the pairs
D

array([[0.82421549, 0.32755369, 0.33198071],
       [0.72642889, 0.72506609, 0.17058938]])

In [3]:
# import libraries
import torch, torchvision
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
#cfg.DATASETS.TRAIN = ("straindata",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 900    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
cfg.MODEL.WEIGHTS = "C:\\Users\\siraj\\UB_prototypes\\Fish Mortality\\fish_detectionv4.pth"  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9   # set a custom testing threshold
cfg.MODEL.DEVICE = 'cpu'
# cfg.DATASETS.TEST=("straindata49",)
predictor = DefaultPredictor(cfg)

In [4]:
from detectron2.utils.visualizer4 import ColorMode
from detectron2.data.datasets import register_coco_instances
register_coco_instances("traindata1",{},"C:\\Users\\siraj\\UB_prototypes\\Biomass_Estimation_prod\\detectFish_v3\\trainval.json","C:\\Users\\siraj\\UB_prototypes\\Biomass_Estimation_prod\\detectFish_v3\\images")
sample_metaadata=MetadataCatalog.get("traindata1") 
MetadataCatalog.get("traindata1").set(thing_classes=["Fish"])
#sample_metaadata=MetadataCatalog.get("traindata1") 

namespace(name='traindata1',
          json_file='C:\\Users\\siraj\\UB_prototypes\\Biomass_Estimation_prod\\detectFish_v3\\trainval.json',
          image_root='C:\\Users\\siraj\\UB_prototypes\\Biomass_Estimation_prod\\detectFish_v3\\images',
          evaluator_type='coco',
          thing_classes=['Fish'])

In [5]:
import math
from PIL import Image
# initialize our centroid tracker and frame dimensions
ct = CentroidTracker()
(H, W) = (None, None)
 
video = cv2.VideoCapture("C:\\Users\\siraj\\office_work_UB\\Testing.mp4")

if (video.isOpened() == False):
    print("Error reading video file")
frame_width = int(video.get(3))
frame_height = int(video.get(4))
size = (frame_width, frame_height)

def predict_frame(frame):
    frame2 = frame.copy()
    #frame2=cv2.resize(frame2,size)
    outputs = predictor(frame2)
    v = Visualizer(frame2[:, :, ::-1],
                   metadata=sample_metaadata, 
                   scale=0.8, 
        instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
        )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    outputnump=outputs["instances"].pred_boxes.tensor.numpy()
    my_class_names=(outputs["instances"].pred_classes.tolist())
    length=len(outputnump)
    print("Length"+str(length))
    print("My classes names")
    print(my_class_names)
    Person=0
    for j in range(len(my_class_names)):
        if(my_class_names[j]==0):
            Person+=1

    for i in range(len(outputnump)):
        box=outputnump[i]
        rects.append(box.astype("int"))
        (startX, startY, endX, endY) = box.astype("int")
        cv2.rectangle(frame, (startX, startY), (endX, endY),
                    (255, 0, 0), 2)    

    myresult=out.get_image()[:, :, ::-1]
    #frame2=cv2.resize(myresult,size)
    #print( "myresult", frame2.shape[1], frame2.shape[0])
    #cv2.putText(frame2, "Total= "+str(Person), (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)
    #cv2.putText(frame2, "Skeleton = "+str(skull), (30, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)
    #cv2.putText(frame2, "Kingfish = "+str(kFish), (30, 140), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 0, 0), 2)
    #cv2.imwrite("yellow_bucket2//frame"+str(abcd)+".jpg",frame2)
    return frame2,rects,length

result = cv2.VideoWriter('fish_tracking4.mp4',
                         cv2.VideoWriter_fourcc(*'DIVX'),
                         30, size)
count = 0
total_dic={}

while(True) and count<=20:
    ret, frame = video.read()
    if ret == True:
        rects = []  
        print(count)
        #frame,rects,length = predict_frame(frame)
        
        frame,rects,length = predict_frame(frame)
        objects = ct.update(rects)
        # loop over the tracked objects
        # loop over the tracked objects
        for (objectID, centroid) in objects.items():
            # draw both the ID of the object and the centroid of the
            # object on the output frame
            #print("Rectsss", rects[objectID])
        
            #frame = cv2.rectangle(frame, (x1_text,y1_text), (x2_text,y2_text), (0,0,255), 2)
            text = "ID {}".format(objectID)
            cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0 , 255), 2)
            cv2.circle(frame, (centroid[0], centroid[1]), 2, (0, 255, 0), -1)
            # show the output frame
                          
        result.write(frame)
        Person=0
        if cv2.waitKey(1) & 0xFF == ord('s'):
            break
        count += 1
    else:
        break         

cv2.destroyAllWindows()
video.release()
result.release() 

0


C:\Users\siraj\anaconda3\envs\detectron2\lib\site-packages\torch\_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ..\aten\src\ATen\native\BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
C:\Users\siraj\anaconda3\envs\detectron2\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[0.85  0.325 0.098]
[1.    0.667 0.5  ]
[1.    0.667 1.   ]
[1.    0.333 0.   ]
[0.333 0.667 0.5  ]
[0.    0.333 0.   ]
[0.85  0.325 0.098]
[0. 1. 0.]
Length8
My classes names
[0, 0, 0, 0, 0, 0, 0, 0]
1
[0. 1. 1.]
[0.    0.167 0.   ]
[0.667 0.333 0.5  ]
[1. 0. 0.]
[0.667 1.    0.   ]
[1.    0.333 0.   ]
[0.667 0.667 1.   ]
Length7
My classes names
[0, 0, 0, 0, 0, 0, 0]
2
[0.667 0.    1.   ]
[0.    0.667 0.   ]
[0.667 0.    1.   ]
[0.    0.667 0.   ]
[0.    0.333 0.   ]
[0.    0.667 0.   ]
[0.5 0.  0. ]
[1. 0. 1.]
Length8
My classes names
[0, 0, 0, 0, 0, 0, 0, 0]
3
[0.333 1.    0.   ]
[0.857 0.857 0.857]
[0.929 0.694 0.125]
[0. 1. 0.]
[0.494 0.184 0.556]
[1.  1.  0.5]
[0.667 1.    0.   ]
[1. 0. 1.]
[0.635 0.078 0.184]
Length9
My classes names
[0, 0, 0, 0, 0, 0, 0, 0, 0]
4
[0.6 0.6 0.6]
[0. 0. 1.]
[0.    0.333 0.   ]
[0.85  0.325 0.098]
[0.    0.333 0.   ]
[1.    0.667 1.   ]
[0. 0. 0.]
Length7
My classes names
[0, 0, 0, 0, 0, 0, 0]
5
[0.667 0.    1.   ]
[0.3 0.3 0.3]
[1.  0.  0.5]
[0.33

In [ ]:
#END